# Préparation des variables de mode de vie

In [117]:
library(tidyverse)
library(factoextra)

# Charger les données
DataPilot <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/01_pilot_2025.rds")
DensityData <- readRDS("../../_SharedFolder_datagotchi_federal_2024/data/clustering/can_2025/density_data.rds")

# Charger le fichier des variables
source("01-1_variables.R")

df_mode_de_vie <- DataPilot %>%
  select(all_of(variables_mode_de_vie))

In [118]:
library(dplyr)

df_mode_de_vie %>%
  select(starts_with("lifestyle_typeTransport")) %>%
  summarise(across(everything(), \(x) sum(x, na.rm = TRUE)))
#  summarise(across(everything(), sum, na.rm = TRUE))

lifestyle_typeTransportCar,lifestyle_typeTransportSUV,lifestyle_typeTransportMoto,lifestyle_typeTransportWalk,lifestyle_typeTransportBicycle,lifestyle_typeTransportPublicTransit
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
598,204,6,88,11,111


In [119]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_postalCode 

4J1 A0E A0G A0M A0P A1B A1E A2H B0E B0K B0P B1T B3H B3J B3M B3S B4A B4E B4N B4P 
  1   1   2   1   1   2   2   1   1   1   1   1   1   1   1   1   1   1   1   2 
B4V C1A C1C C1E G0A G0C G0G G0K G0M G0R G0S G0W G0X G0l G0s G1B G1H G1T G1V G1W 
  2   1   1   1   2   3   1   1   1   1   1   2   1   2   2   2   2   1   2   2 
G1Y G1h G1w G1x G2B G2j G3A G3E G3H G3J G3e G3h G4R G4s G4z G5C G5L G5V G5Y G5Z 
  1   2   1   1   1   1   1   1   1   1   1   1   3   1   1   1   2   1   1   1 
G5l G6G G6L G6P G6W G6Y G6e G7A G7G G7H G7P G7S G7X G7p G8B G8P G8V G8W G8Y G8Z 
  1   1   1   2   1   2   1   1   1   1   1   2   2   1   1   1   1   1   1   2 
G8z G9A G9H G9b G9n H1A H1E H1G H1K H1L H1M H1P H1T H1W H1X H1Y H1t H1x H2C H2G 
  1   1   1   1   1   1   1   1   1   1   4   1   1   2   2   3   1   1   1   1 
H2H H2J H2K H2M H2S H2X H2z H3C H3H H3K H3T H3W H3c H3e H3y H4B H4H H4J H4N H4W 
  1   1   2   3   2   1   1   4   1   2   3   1   1   1   1   1   1 

### Transformations

- Regroupement de ses_dwelling
  - ses_dwellingCondo : ses_dwellingCondo + ses_dwellingLoft + ses_dwellingTour
  - ses_dwellingOther : ses_dwellingOther + ses_dwellingCoop + ses_dwellingHLM + ses_dwellingMobile

- Regroupement des act_transport
  - lifestyle_typeTransportCar : lifestyle_typeTransportCar + lifestyle_typeTransportMoto (parce qu'il y avraiment trop peu de moto, assigner la classe la plus populeuse)
  - lifestyle_typeTransportActive:   lifestyle_typeTransportWalk + lifestyle_typeTransportBicycle
  - Les autres ne sont pas regroupés

In [120]:
# Regroupement de ses_dwelling
#-----------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    ses_dwellingCondo = if_else(
      ses_dwellingLoft == 1 | ses_dwellingTour == 1,
      1,
      ses_dwellingCondo
    ),
    ses_dwellingOther = if_else(
      ses_dwellingCoop == 1 | ses_dwellingHLM == 1 | ses_dwellingMobile == 1,
      1,
      ses_dwellingOther
    )
  ) %>%
  select(
    -ses_dwellingLoft,
    -ses_dwellingTour,
    -ses_dwellingCoop,
    -ses_dwellingHLM,
    -ses_dwellingMobile
  )

In [121]:
# Regroupement de act_transport
#------------------------------

df_mode_de_vie <- df_mode_de_vie %>%
  mutate(
    lifestyle_typeTransportCar = if_else(
      lifestyle_typeTransportMoto == 1 | lifestyle_typeTransportSUV == 1,
      1,
      lifestyle_typeTransportCar
    ),
    lifestyle_typeTransportNoCar = if_else(
      lifestyle_typeTransportWalk == 1 | lifestyle_typeTransportBicycle == 1 | lifestyle_typeTransportPublicTransit == 1,
      1,
      0
    )
  ) %>%
  select(
    -lifestyle_typeTransportMoto,
    -lifestyle_typeTransportSUV,
    -lifestyle_typeTransportWalk,
    -lifestyle_typeTransportBicycle,
    -lifestyle_typeTransportPublicTransit)

In [122]:
variables_mode_de_vie_clust <- c(
  "id",
  "ses_postalCode",
  "ses_dwellingApp",
  "ses_dwellingCondo",
  "ses_dwellingDetachedHouse",
  "ses_dwellingTownhouse",
  "ses_dwellingDuplex",
  "ses_dwellingOther",
  "lifestyle_typeTransportCar",
  "lifestyle_typeTransportNoCar"
)

In [123]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_postalCode 

4J1 A0E A0G A0M A0P A1B A1E A2H B0E B0K B0P B1T B3H B3J B3M B3S B4A B4E B4N B4P 
  1   1   2   1   1   2   2   1   1   1   1   1   1   1   1   1   1   1   1   2 
B4V C1A C1C C1E G0A G0C G0G G0K G0M G0R G0S G0W G0X G0l G0s G1B G1H G1T G1V G1W 
  2   1   1   1   2   3   1   1   1   1   1   2   1   2   2   2   2   1   2   2 
G1Y G1h G1w G1x G2B G2j G3A G3E G3H G3J G3e G3h G4R G4s G4z G5C G5L G5V G5Y G5Z 
  1   2   1   1   1   1   1   1   1   1   1   1   3   1   1   1   2   1   1   1 
G5l G6G G6L G6P G6W G6Y G6e G7A G7G G7H G7P G7S G7X G7p G8B G8P G8V G8W G8Y G8Z 
  1   1   1   2   1   2   1   1   1   1   1   2   2   1   1   1   1   1   1   2 
G8z G9A G9H G9b G9n H1A H1E H1G H1K H1L H1M H1P H1T H1W H1X H1Y H1t H1x H2C H2G 
  1   1   1   1   1   1   1   1   1   1   4   1   1   2   2   3   1   1   1   1 
H2H H2J H2K H2M H2S H2X H2z H3C H3H H3K H3T H3W H3c H3e H3y H4B H4H H4J H4N H4W 
  1   1   2   3   2   1   1   4   1   2   3   1   1   1   1   1   1 

In [124]:
# Créer une variable rural/urbain avec les codes postaux
library(dplyr)

# Mettre tous les codes postaux en majuscule
df_mode_de_vie <- df_mode_de_vie %>%
  mutate(ses_postalCode = toupper(ses_postalCode))

# Join pour avoir la classification des codes postaux
df_mode_de_vie <- left_join(df_mode_de_vie, DensityData, by = c("ses_postalCode" = "fsa")) %>%
  rename(ses_urbanity = ses_population_centre)

# Calculer le code postal (FSA) le plus populaire parmi les observations valides
mode_fsa <- df_mode_de_vie %>%
  filter(!is.na(ses_urbanity)) %>%
  count(ses_postalCode, sort = TRUE) %>%
  slice(1) %>%
  pull(ses_postalCode)

# Imputer le mode pour les codes non appariés
df_mode_de_vie <- df_mode_de_vie %>%
  mutate(ses_postalCode = if_else(is.na(ses_urbanity), mode_fsa, ses_postalCode))

# Supprimer l'ancienne variable
df_mode_de_vie <- df_mode_de_vie %>% select(-ses_urbanity)

# Refaire le join pour mettre à jour le matching avec le code postal imputé
df_mode_de_vie <- left_join(df_mode_de_vie, DensityData, by = c("ses_postalCode" = "fsa")) %>%
  rename(ses_urbanity = ses_population_centre)

# Vérifier le résultat
table(df_mode_de_vie$ses_urbanity, useNA = "ifany")

#Créer la variable binaire
df_mode_de_vie$ses_urban <- NA
df_mode_de_vie$ses_urban[df_mode_de_vie$ses_urbanity == "population_centre"] <- 1
df_mode_de_vie$ses_urban[df_mode_de_vie$ses_urbanity != "population_centre"] <- 0
table(df_mode_de_vie$ses_urban, useNA = "ifany") 


population_centre             rural 
              648               373 


  0   1 
373 648 

In [125]:

names(df_mode_de_vie)
# Sauvegarder les données préparées
saveRDS(df_mode_de_vie, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/02_pilot_2025_mode_de_vie.rds")

[1] "id"                           "ses_postalCode"              
 [3] "ses_dwellingApp"              "ses_dwellingCondo"           
 [5] "ses_dwellingDetachedHouse"    "ses_dwellingTownhouse"       
 [7] "ses_dwellingDuplex"           "ses_dwellingOther"           
 [9] "lifestyle_typeTransportCar"   "lifestyle_typeTransportNoCar"
[11] "population.x"                 "area_km2.x"                  
[13] "density_by_km2.x"             "province.x"                  
[15] "population.y"                 "area_km2.y"                  
[17] "density_by_km2.y"             "province.y"                  
[19] "ses_urbanity"                 "ses_urban"

---

# SUGGESTION : Activité de Clustering

Si souhaité, effectuer un exercice de clustering avec ce sous-groupe de variables pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.